## installing libs

In [25]:
!pip install -q ultralytics roboflow

In [26]:
from roboflow import Roboflow

In [27]:
!pip install -q gdown inference-gpu

^C
ERROR: Operation cancelled by user


## get the key point detection from roboflow

In [28]:
from inference import get_model
FIELD_DETECTION_MODEL_ID = "football-field-detection-f07vi/14"
FIELD_DETECTION_MODEL = get_model(model_id=FIELD_DETECTION_MODEL_ID, api_key="5lupAJ0ZuQ3Z3DH8ic8g")

2024-09-15 18:52:08.340825833 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:640 CreateExecutionProviderInstance] Failed to create CUDAExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements to ensure all dependencies are met.


KeyboardInterrupt: 

In [ ]:
# rf = Roboflow(api_key="5lupAJ0ZuQ3Z3DH8ic8g")

# project = rf.workspace("wisd-projects").project("football-analysis-pwwi6")
# version = project.version(1)
# # dataset = version.download("yolov8")

In [ ]:
# !sed -i 's|\(train: \).*|\1../train/images|' {dataset.location}/data.yaml
# !sed -i 's|\(val: \).*|\1../valid/images|' {dataset.location}/data.yaml

In [ ]:
# # import shutil 
# # shutil.rmtree("/kaggle/working/runs")
# import os

# # Disable W&B completely
# os.environ["WANDB_MODE"] = "disabled"

In [ ]:
# !yolo task=detect mode=train model=yolov8x.pt data={dataset.location}/data.yaml batch=4 epochs=100 imgsz=1280

In [ ]:
# !yolo task=detect mode=val model=/kaggle/working/runs/detect/train4/weights/best.pt data={dataset.location}/data.yaml imgsz=1280

In [ ]:
# !yolo predict model=/kaggle/working/runs/detect/train4/weights/best.pt source="/kaggle/input/video1/08fd33_4.mp4" imgsz=1280 save=True

In [ ]:
# project.version(1).deploy(model_type="yolov8", model_path="/kaggle/working/runs/detect/train4/")

## using the model for detection players,goalkeeper,referee and ball

In [ ]:
import cv2
# get the total number of frames 
def get_number_of_frames(VIDEO_SRC):
    cap = cv2.VideoCapture(VIDEO_SRC)
    if not cap.isOpened():
        return -1
    else:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        return total_frames,fps

In [ ]:
!pip install -q supervision

In [ ]:
import cv2
import supervision as sv

# drawing a rectaggle showing the ball_control percentage for each team
def draw_team_ball_control(frame,ball_control,MODEL_CLASSES):
        
        overlay = frame.copy()
        cv2.rectangle(overlay, (1210, 145), (1838,70), (255,255,255), -1 )
        alpha = 0.6
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

        team_1 = ball_control[MODEL_CLASSES["team1"]] / (ball_control[MODEL_CLASSES["team1"]]+ball_control[MODEL_CLASSES["team2"]])
        team_2 = ball_control[MODEL_CLASSES["team2"]] / (ball_control[MODEL_CLASSES["team1"]]+ball_control[MODEL_CLASSES["team2"]])

        cv2.putText(frame,"Ball Control : ",(1220,115),cv2.FONT_HERSHEY_DUPLEX, 1,(0,0,0),3)
        cv2.putText(frame, f"{team_1*100:.2f}%",(1500,115), cv2.FONT_HERSHEY_SIMPLEX, 1, sv.Color.as_bgr(sv.Color.from_hex('#1E90FF')), 3)
        cv2.putText(frame, f"{team_2*100:.2f}%",(1700,115), cv2.FONT_HERSHEY_SIMPLEX, 1, sv.Color.as_bgr(sv.Color.from_hex('#DC143C')), 3)

        return frame

In [ ]:
# making some custem annotations

import supervision as sv

colors = {
    "team1":sv.ColorPalette.from_hex(['#1E90FF']),
    "team2":sv.ColorPalette.from_hex(['#DC143C']),
    "referee":sv.ColorPalette.from_hex(['#FFD700']),
    "goalkepper":sv.ColorPalette.from_hex(['#FFF']),
    "label_text":sv.Color.from_hex('#000'),
    "ball":sv.Color.from_hex('#FF8C00'),
    "active_player":sv.Color.from_rgb_tuple((255,0,0))
}

LABEL_TEXT_POSITION = sv.Position.BOTTOM_CENTER


team1_ellipse_annotator = sv.EllipseAnnotator(color=colors["team1"])  
team2_ellipse_annotator = sv.EllipseAnnotator(color=colors['team2'])
referee_ellipse_annotator = sv.EllipseAnnotator(color=colors['referee'])
goalkepper_ellipse_annotator = sv.EllipseAnnotator(color=colors['goalkepper'])
active_player_annotator = sv.TriangleAnnotator(color=colors['active_player'],base=18,height=18)

team1_label_annotator = sv.LabelAnnotator(color=colors['team1'],text_color=colors['label_text'], text_position=LABEL_TEXT_POSITION)
team2_label_annotator = sv.LabelAnnotator(color=colors['team2'],text_color=colors['label_text'],text_position=LABEL_TEXT_POSITION)
referee_label_annotator = sv.LabelAnnotator(color=colors['referee'],text_color=colors['label_text'],border_radius=30, text_position=LABEL_TEXT_POSITION)
goalkepper_label_annotator = sv.LabelAnnotator(color=colors['goalkepper'],text_color=colors['label_text'],border_radius=30,text_position=LABEL_TEXT_POSITION)

ball_triangle_annotator = sv.TriangleAnnotator(color=colors['ball'], base=18, height=18)


# annotate the frames
def annotate_frames(frame,all_detection,labels,ball_posession,MODEL_CLASSES):
    annotated_frame = frame.copy()
    annotated_frame = team1_ellipse_annotator.annotate(scene=annotated_frame, detections=all_detection["team1"])
    annotated_frame = team2_ellipse_annotator.annotate(scene=annotated_frame, detections=all_detection["team2"])
    annotated_frame = referee_ellipse_annotator.annotate(scene=annotated_frame, detections=all_detection["referee"])
    annotated_frame = goalkepper_ellipse_annotator.annotate(scene=annotated_frame,detections=all_detection["goalkeepers"])
    annotated_frame = ball_triangle_annotator.annotate(scene=annotated_frame, detections=all_detection["ball"])
    annotated_frame = team1_label_annotator.annotate(scene=annotated_frame, detections=all_detection["team1"], labels=labels["labels_team1"])
    annotated_frame = team2_label_annotator.annotate(scene=annotated_frame, detections=all_detection["team2"], labels=labels["labels_team2"])
    annotated_frame = referee_label_annotator.annotate(scene=annotated_frame,detections=all_detection["referee"],labels=labels["labels_referee"])
    annotated_frame = goalkepper_label_annotator.annotate(scene=annotated_frame,detections=all_detection["goalkeepers"],labels=labels["labels_gk"])
    annotated_frame = active_player_annotator.annotate(scene=annotated_frame,detections=all_detection["active_player"]) 
    annotated_frame = draw_team_ball_control(annotated_frame,ball_posession,MODEL_CLASSES)
    return annotated_frame

###  team assigner

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import supervision as sv
import numpy as np
from PIL import Image,ImageOps
import cv2
class Assigner:
    def __init__(self) -> None:
         self.team_colors={}

    def get_clustering_model(self,image):
        image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        image_2d = image.reshape(-1,3)
        # Preform K-means with 2 clusters
        kmeans = KMeans(n_clusters=2,n_init=1)
        kmeans.fit(image_2d)

        return kmeans

    def get_player_team(self,frame, player_bbox,kmeans):
            player_color = self.get_player_color(frame, player_bbox)
            team_id = kmeans.predict(player_color.reshape(1, -1))[0]
            return team_id

    def get_player_color(self,frame, bbox):
        image = frame[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        top_half_image = image[0:int(image.shape[0] / 2), :]
        kmeans = self.get_clustering_model(top_half_image)
        labels = kmeans.labels_

        clustered_image = labels.reshape(top_half_image.shape[0], top_half_image.shape[1])
        corner_clusters = [clustered_image[0, 0], clustered_image[0, -1], clustered_image[-1, 0], clustered_image[-1, -1]]
        non_player_cluster = max(set(corner_clusters), key=corner_clusters.count)
        player_cluster = 1 - non_player_cluster
        player_color = kmeans.cluster_centers_[player_cluster]

        # centriod = kmeans.cluster_centers_
        # percent = []
        # labels = list(labels)
        # for i in range(len(centriod)):
        #      j = labels.count(i)
        #      j/=len(labels)
        #      percent.append(j)
        
        # plt.pie(percent,colors=np.array(centriod/255),labels=np.arange(len(centriod)))
        
        # plt.show()
        # print(player_cluster)
        # sv.plot_image(clustered_image)
        return player_color

    def assign_team_color(self,frame, players_detections):
            
            player_colors = []
            for xy2 in players_detections.xyxy:
                bbox = xy2
                player_color = self.get_player_color(frame, bbox)
                player_colors.append(player_color)
            
            kmeans = KMeans(n_clusters=2, init="k-means++", n_init="auto")
            kmeans.fit(player_colors)
            self.team_colors[1] = kmeans.cluster_centers_[0]
            self.team_colors[2] = kmeans.cluster_centers_[1]
            return kmeans

### assign the ball to a player

In [ ]:
import numpy as np


MAX_DISTANCE = 70
    

def get_center_of_bbox(bbox):
    x1,y1,x2,y2 = bbox
    return int((x1+x2)/2),int((y1+y2)/2)

def measure_distance(p1,p2):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)


def assign_ball_to_player(players_detections,ball_bbox):
    if len(ball_bbox)==0 or len(players_detections.xyxy) == 0:
        return -1
    ball_position = get_center_of_bbox(ball_bbox[0])

    miniumum_distance = 99999
    assigned_player=-1

    for object_ind , _ in enumerate(players_detections.class_id):
        player_bbox = players_detections.xyxy[object_ind]

        distance_left = measure_distance((player_bbox[0],player_bbox[-1]),ball_position)
        distance_right = measure_distance((player_bbox[2],player_bbox[-1]),ball_position)
        distance = min(distance_left,distance_right)

        if distance < MAX_DISTANCE:
            if distance < miniumum_distance:
                miniumum_distance = distance
                assigned_player = object_ind

    return assigned_player

### this code is from the roboflow repo link -> https://github.com/roboflow/sports 

In [ ]:
from dataclasses import dataclass, field
from typing import List, Tuple


@dataclass
class SoccerPitchConfiguration:
    width: int = 7000  # [cm]
    length: int = 12000  # [cm]
    penalty_box_width: int = 4100  # [cm]
    penalty_box_length: int = 2015  # [cm]
    goal_box_width: int = 1832  # [cm]
    goal_box_length: int = 550  # [cm]
    centre_circle_radius: int = 915  # [cm]
    penalty_spot_distance: int = 1100  # [cm]

    @property
    def vertices(self) -> List[Tuple[int, int]]:
        return [
            (0, 0),  # 1
            (0, (self.width - self.penalty_box_width) / 2),  # 2
            (0, (self.width - self.goal_box_width) / 2),  # 3
            (0, (self.width + self.goal_box_width) / 2),  # 4
            (0, (self.width + self.penalty_box_width) / 2),  # 5
            (0, self.width),  # 6
            (self.goal_box_length, (self.width - self.goal_box_width) / 2),  # 7
            (self.goal_box_length, (self.width + self.goal_box_width) / 2),  # 8
            (self.penalty_spot_distance, self.width / 2),  # 9
            (self.penalty_box_length, (self.width - self.penalty_box_width) / 2),  # 10
            (self.penalty_box_length, (self.width - self.goal_box_width) / 2),  # 11
            (self.penalty_box_length, (self.width + self.goal_box_width) / 2),  # 12
            (self.penalty_box_length, (self.width + self.penalty_box_width) / 2),  # 13
            (self.length / 2, 0),  # 14
            (self.length / 2, self.width / 2 - self.centre_circle_radius),  # 15
            (self.length / 2, self.width / 2 + self.centre_circle_radius),  # 16
            (self.length / 2, self.width),  # 17
            (
                self.length - self.penalty_box_length,
                (self.width - self.penalty_box_width) / 2
            ),  # 18
            (
                self.length - self.penalty_box_length,
                (self.width - self.goal_box_width) / 2
            ),  # 19
            (
                self.length - self.penalty_box_length,
                (self.width + self.goal_box_width) / 2
            ),  # 20
            (
                self.length - self.penalty_box_length,
                (self.width + self.penalty_box_width) / 2
            ),  # 21
            (self.length - self.penalty_spot_distance, self.width / 2),  # 22
            (
                self.length - self.goal_box_length,
                (self.width - self.goal_box_width) / 2
            ),  # 23
            (
                self.length - self.goal_box_length,
                (self.width + self.goal_box_width) / 2
            ),  # 24
            (self.length, 0),  # 25
            (self.length, (self.width - self.penalty_box_width) / 2),  # 26
            (self.length, (self.width - self.goal_box_width) / 2),  # 27
            (self.length, (self.width + self.goal_box_width) / 2),  # 28
            (self.length, (self.width + self.penalty_box_width) / 2),  # 29
            (self.length, self.width),  # 30
            (self.length / 2 - self.centre_circle_radius, self.width / 2),  # 31
            (self.length / 2 + self.centre_circle_radius, self.width / 2),  # 32
        ]

    edges: List[Tuple[int, int]] = field(default_factory=lambda: [
        (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (7, 8),
        (10, 11), (11, 12), (12, 13), (14, 15), (15, 16),
        (16, 17), (18, 19), (19, 20), (20, 21), (23, 24),
        (25, 26), (26, 27), (27, 28), (28, 29), (29, 30),
        (1, 14), (2, 10), (3, 7), (4, 8), (5, 13), (6, 17),
        (14, 25), (18, 26), (23, 27), (24, 28), (21, 29), (17, 30)
    ])

    labels: List[str] = field(default_factory=lambda: [
        "01", "02", "03", "04", "05", "06", "07", "08", "09", "10",
        "11", "12", "13", "15", "16", "17", "18", "20", "21", "22",
        "23", "24", "25", "26", "27", "28", "29", "30", "31", "32",
        "14", "19"
    ])

    colors: List[str] = field(default_factory=lambda: [
        "#FF1493", "#FF1493", "#FF1493", "#FF1493", "#FF1493", "#FF1493",
        "#FF1493", "#FF1493", "#FF1493", "#FF1493", "#FF1493", "#FF1493",
        "#FF1493", "#00BFFF", "#00BFFF", "#00BFFF", "#00BFFF", "#FF6347",
        "#FF6347", "#FF6347", "#FF6347", "#FF6347", "#FF6347", "#FF6347",
        "#FF6347", "#FF6347", "#FF6347", "#FF6347", "#FF6347", "#FF6347",
        "#00BFFF", "#00BFFF"
    ])


In [ ]:
from typing import Optional, List

import cv2
import supervision as sv
import numpy as np



def draw_pitch(
    config: SoccerPitchConfiguration,
    background_color: sv.Color = sv.Color(34, 139, 34),
    line_color: sv.Color = sv.Color.WHITE,
    padding: int = 50,
    line_thickness: int = 4,
    point_radius: int = 8,
    scale: float = 0.1
) -> np.ndarray:
    """
    Draws a soccer pitch with specified dimensions, colors, and scale.

    Args:
        config (SoccerPitchConfiguration): Configuration object containing the
            dimensions and layout of the pitch.
        background_color (sv.Color, optional): Color of the pitch background.
            Defaults to sv.Color(34, 139, 34).
        line_color (sv.Color, optional): Color of the pitch lines.
            Defaults to sv.Color.WHITE.
        padding (int, optional): Padding around the pitch in pixels.
            Defaults to 50.
        line_thickness (int, optional): Thickness of the pitch lines in pixels.
            Defaults to 4.
        point_radius (int, optional): Radius of the penalty spot points in pixels.
            Defaults to 8.
        scale (float, optional): Scaling factor for the pitch dimensions.
            Defaults to 0.1.

    Returns:
        np.ndarray: Image of the soccer pitch.
    """
    scaled_width = int(config.width * scale)
    scaled_length = int(config.length * scale)
    scaled_circle_radius = int(config.centre_circle_radius * scale)
    scaled_penalty_spot_distance = int(config.penalty_spot_distance * scale)

    pitch_image = np.ones(
        (scaled_width + 2 * padding,
         scaled_length + 2 * padding, 3),
        dtype=np.uint8
    ) * np.array(background_color.as_bgr(), dtype=np.uint8)

    for start, end in config.edges:
        point1 = (int(config.vertices[start - 1][0] * scale) + padding,
                  int(config.vertices[start - 1][1] * scale) + padding)
        point2 = (int(config.vertices[end - 1][0] * scale) + padding,
                  int(config.vertices[end - 1][1] * scale) + padding)
        cv2.line(
            img=pitch_image,
            pt1=point1,
            pt2=point2,
            color=line_color.as_bgr(),
            thickness=line_thickness
        )

    centre_circle_center = (
        scaled_length // 2 + padding,
        scaled_width // 2 + padding
    )
    cv2.circle(
        img=pitch_image,
        center=centre_circle_center,
        radius=scaled_circle_radius,
        color=line_color.as_bgr(),
        thickness=line_thickness
    )

    penalty_spots = [
        (
            scaled_penalty_spot_distance + padding,
            scaled_width // 2 + padding
        ),
        (
            scaled_length - scaled_penalty_spot_distance + padding,
            scaled_width // 2 + padding
        )
    ]
    for spot in penalty_spots:
        cv2.circle(
            img=pitch_image,
            center=spot,
            radius=point_radius,
            color=line_color.as_bgr(),
            thickness=-1
        )

    return pitch_image


def draw_points_on_pitch(
    config: SoccerPitchConfiguration,
    xy: np.ndarray,
    face_color: sv.Color = sv.Color.RED,
    edge_color: sv.Color = sv.Color.BLACK,
    radius: int = 10,
    thickness: int = 2,
    padding: int = 50,
    scale: float = 0.1,
    pitch: Optional[np.ndarray] = None
) -> np.ndarray:
    """
    Draws points on a soccer pitch.

    Args:
        config (SoccerPitchConfiguration): Configuration object containing the
            dimensions and layout of the pitch.
        xy (np.ndarray): Array of points to be drawn, with each point represented by
            its (x, y) coordinates.
        face_color (sv.Color, optional): Color of the point faces.
            Defaults to sv.Color.RED.
        edge_color (sv.Color, optional): Color of the point edges.
            Defaults to sv.Color.BLACK.
        radius (int, optional): Radius of the points in pixels.
            Defaults to 10.
        thickness (int, optional): Thickness of the point edges in pixels.
            Defaults to 2.
        padding (int, optional): Padding around the pitch in pixels.
            Defaults to 50.
        scale (float, optional): Scaling factor for the pitch dimensions.
            Defaults to 0.1.
        pitch (Optional[np.ndarray], optional): Existing pitch image to draw points on.
            If None, a new pitch will be created. Defaults to None.

    Returns:
        np.ndarray: Image of the soccer pitch with points drawn on it.
    """
    if pitch is None:
        pitch = draw_pitch(
            config=config,
            padding=padding,
            scale=scale
        )

    for point in xy:
        scaled_point = (
            int(point[0] * scale) + padding,
            int(point[1] * scale) + padding
        )
        cv2.circle(
            img=pitch,
            center=scaled_point,
            radius=radius,
            color=face_color.as_bgr(),
            thickness=-1
        )
        cv2.circle(
            img=pitch,
            center=scaled_point,
            radius=radius,
            color=edge_color.as_bgr(),
            thickness=thickness
        )

    return pitch


def draw_paths_on_pitch(
    config: SoccerPitchConfiguration,
    paths: List[np.ndarray],
    color: sv.Color = sv.Color.WHITE,
    thickness: int = 2,
    padding: int = 50,
    scale: float = 0.1,
    pitch: Optional[np.ndarray] = None
) -> np.ndarray:
    """
    Draws paths on a soccer pitch.

    Args:
        config (SoccerPitchConfiguration): Configuration object containing the
            dimensions and layout of the pitch.
        paths (List[np.ndarray]): List of paths, where each path is an array of (x, y)
            coordinates.
        color (sv.Color, optional): Color of the paths.
            Defaults to sv.Color.WHITE.
        thickness (int, optional): Thickness of the paths in pixels.
            Defaults to 2.
        padding (int, optional): Padding around the pitch in pixels.
            Defaults to 50.
        scale (float, optional): Scaling factor for the pitch dimensions.
            Defaults to 0.1.
        pitch (Optional[np.ndarray], optional): Existing pitch image to draw paths on.
            If None, a new pitch will be created. Defaults to None.

    Returns:
        np.ndarray: Image of the soccer pitch with paths drawn on it.
    """
    if pitch is None:
        pitch = draw_pitch(
            config=config,
            padding=padding,
            scale=scale
        )

    for path in paths:
        scaled_path = [
            (
                int(point[0] * scale) + padding,
                int(point[1] * scale) + padding
            )
            for point in path if point.size > 0
        ]

        if len(scaled_path) < 2:
            continue

        for i in range(len(scaled_path) - 1):
            cv2.line(
                img=pitch,
                pt1=scaled_path[i],
                pt2=scaled_path[i + 1],
                color=color.as_bgr(),
                thickness=thickness
            )

        return pitch


def draw_pitch_voronoi_diagram(
    config: SoccerPitchConfiguration,
    team_1_xy: np.ndarray,
    team_2_xy: np.ndarray,
    team_1_color: sv.Color = sv.Color.RED,
    team_2_color: sv.Color = sv.Color.WHITE,
    opacity: float = 0.5,
    padding: int = 50,
    scale: float = 0.1,
    pitch: Optional[np.ndarray] = None
) -> np.ndarray:
    """
    Draws a Voronoi diagram on a soccer pitch representing the control areas of two
    teams.

    Args:
        config (SoccerPitchConfiguration): Configuration object containing the
            dimensions and layout of the pitch.
        team_1_xy (np.ndarray): Array of (x, y) coordinates representing the positions
            of players in team 1.
        team_2_xy (np.ndarray): Array of (x, y) coordinates representing the positions
            of players in team 2.
        team_1_color (sv.Color, optional): Color representing the control area of
            team 1. Defaults to sv.Color.RED.
        team_2_color (sv.Color, optional): Color representing the control area of
            team 2. Defaults to sv.Color.WHITE.
        opacity (float, optional): Opacity of the Voronoi diagram overlay.
            Defaults to 0.5.
        padding (int, optional): Padding around the pitch in pixels.
            Defaults to 50.
        scale (float, optional): Scaling factor for the pitch dimensions.
            Defaults to 0.1.
        pitch (Optional[np.ndarray], optional): Existing pitch image to draw the
            Voronoi diagram on. If None, a new pitch will be created. Defaults to None.

    Returns:
        np.ndarray: Image of the soccer pitch with the Voronoi diagram overlay.
    """
    if pitch is None:
        pitch = draw_pitch(
            config=config,
            padding=padding,
            scale=scale
        )

    scaled_width = int(config.width * scale)
    scaled_length = int(config.length * scale)

    voronoi = np.zeros_like(pitch, dtype=np.uint8)

    team_1_color_bgr = np.array(team_1_color.as_bgr(), dtype=np.uint8)
    team_2_color_bgr = np.array(team_2_color.as_bgr(), dtype=np.uint8)

    y_coordinates, x_coordinates = np.indices((
        scaled_width + 2 * padding,
        scaled_length + 2 * padding
    ))

    y_coordinates -= padding
    x_coordinates -= padding

    def calculate_distances(xy, x_coordinates, y_coordinates):
        return np.sqrt((xy[:, 0][:, None, None] * scale - x_coordinates) ** 2 +
                       (xy[:, 1][:, None, None] * scale - y_coordinates) ** 2)

    distances_team_1 = calculate_distances(team_1_xy, x_coordinates, y_coordinates)
    distances_team_2 = calculate_distances(team_2_xy, x_coordinates, y_coordinates)

    min_distances_team_1 = np.min(distances_team_1, axis=0)
    min_distances_team_2 = np.min(distances_team_2, axis=0)

    control_mask = min_distances_team_1 < min_distances_team_2

    voronoi[control_mask] = team_1_color_bgr
    voronoi[~control_mask] = team_2_color_bgr

    overlay = cv2.addWeighted(voronoi, opacity, pitch, 1 - opacity, 0)

    return overlay


In [ ]:
from typing import Tuple
import cv2
import numpy as np
import numpy.typing as npt


class ViewTransformer:
    def __init__(
            self,
            source: npt.NDArray[np.float32],
            target: npt.NDArray[np.float32]
    ) -> None:
        """
        Initialize the ViewTransformer with source and target points.

        Args:
            source (npt.NDArray[np.float32]): Source points for homography calculation.
            target (npt.NDArray[np.float32]): Target points for homography calculation.

        Raises:
            ValueError: If source and target do not have the same shape or if they are
                not 2D coordinates.
        """
        if source.shape != target.shape:
            raise ValueError("Source and target must have the same shape.")
        if source.shape[1] != 2:
            raise ValueError("Source and target points must be 2D coordinates.")

        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m, _ = cv2.findHomography(source, target)
        if self.m is None:
            raise ValueError("Homography matrix could not be calculated.")

    def transform_points(
            self,
            points: npt.NDArray[np.float32]
    ) -> npt.NDArray[np.float32]:
        """
        Transform the given points using the homography matrix.

        Args:
            points (npt.NDArray[np.float32]): Points to be transformed.

        Returns:
            npt.NDArray[np.float32]: Transformed points.

        Raises:
            ValueError: If points are not 2D coordinates.
        """
        if points.size == 0:
            return points

        if points.shape[1] != 2:
            raise ValueError("Points must be 2D coordinates.")

        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2).astype(np.float32)

    def transform_image(
            self,
            image: npt.NDArray[np.uint8],
            resolution_wh: Tuple[int, int]
    ) -> npt.NDArray[np.uint8]:
        """
        Transform the given image using the homography matrix.

        Args:
            image (npt.NDArray[np.uint8]): Image to be transformed.
            resolution_wh (Tuple[int, int]): Width and height of the output image.

        Returns:
            npt.NDArray[np.uint8]: Transformed image.

        Raises:
            ValueError: If the image is not either grayscale or color.
        """
        if len(image.shape) not in {2, 3}:
            raise ValueError("Image must be either grayscale or color.")
        return cv2.warpPerspective(image, self.m, resolution_wh)


### main 

In [30]:
import numpy as np
from ultralytics import YOLO
import supervision as sv
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle

# static variables
VIDEO_SRC = "/kaggle/input/video1/08fd33_4.mp4"
OUT_VIDEO = "/kaggle/working/08fd33_4_output.mp4" 
MODEL_SRC = "/kaggle/working/runs/detect/train4/weights/best.pt"
MODEL_CLASSES = {
     "ball":0,
     "goalkepper":1,
     "player":2,
     "referee":3,
     "team1":4,
     "team2":5,
     "active_player":6
}

CONFIG = SoccerPitchConfiguration()

def main():
    model = YOLO(MODEL_SRC)
    
    # get the total frames
    total_frames , fps = get_number_of_frames(VIDEO_SRC)

    frame_generator = sv.get_video_frames_generator(VIDEO_SRC)

    # Get the first frame to determine the video dimensions
    first_frame = next(frame_generator)
    height, width, _ = first_frame.shape

    # Initialize the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUT_VIDEO, fourcc, fps, (width, height+800))
    
    # 
    tracker = sv.ByteTrack()
    tracker.reset()
    tracker1 = sv.ByteTrack()
    tracker1.reset()


    # Process each frame in the video
    is_first_frame = True
    class_id_active_player = None
    kmeans = None
    ball_posession = {MODEL_CLASSES["team1"]:0,MODEL_CLASSES["team2"]:0}
    team_colors={}
    ########################## for debuging
    i = 1
    ##########################
 
    for frame in tqdm(frame_generator,total=total_frames):
        
        # try:
            # Predict the boxes with the model
            result = model.predict(frame, conf=0.3,verbose=False)[0]
            detections = sv.Detections.from_ultralytics(result)
            
            # 
            goalkeepers_detections = detections[detections.class_id == MODEL_CLASSES["goalkepper"]]
            
            # convert goalkeeper to player
#             for object_ind , class_id in enumerate(detections.class_id):
#                 if class_id == MODEL_CLASSES["goalkepper"]:
#                     detections.class_id[object_ind] = MODEL_CLASSES["player"]

            ball_detections = detections[detections.class_id == MODEL_CLASSES["ball"]]
            players_detections = detections[detections.class_id == MODEL_CLASSES["player"]]
            referee_detections = detections[detections.class_id == MODEL_CLASSES["referee"]]


            # Appling Non-Maximum Suppression (NMS) to the players detections
            players_detections = players_detections.with_nms(threshold=0.5)
            
            # Assign Player Teams
            if is_first_frame:
                team_assigner = Assigner()
                kmeans = team_assigner.assign_team_color(frame, players_detections)
                team_colors = team_assigner.team_colors
                is_first_frame = False
            for object_ind , _ in enumerate(players_detections.class_id):
                player_color = team_assigner.get_player_color(frame,players_detections.xyxy[object_ind])
                team_id = team_assigner.get_player_team(frame, players_detections.xyxy[object_ind],kmeans)
                if team_id == 0:
                    players_detections.class_id[object_ind] = MODEL_CLASSES["team1"]
                    a = np.sqrt((player_color[0] - team_colors[1][0])**2 + (player_color[1] - team_colors[1][1])**2 + (player_color[2]- team_colors[1][2])**2)
                    if a > 110 and a < 180:
                        players_detections.class_id[object_ind] = MODEL_CLASSES["goalkepper"]
                elif team_id == 1:
                    players_detections.class_id[object_ind] = MODEL_CLASSES["team2"]
                    a = np.sqrt((player_color[0] - team_colors[2][0])**2 + (player_color[1] - team_colors[2][1])**2 + (player_color[2] - team_colors[2][2])**2)
                    if a > 110 and a < 180:
                        players_detections.class_id[object_ind] = MODEL_CLASSES["goalkepper"]
            
            
            goalkeepers_detections1 = players_detections[players_detections.class_id == MODEL_CLASSES["goalkepper"]]
            goalkeepers_detections = sv.Detections.merge([goalkeepers_detections,goalkeepers_detections1])
            
            team1_detections =  players_detections[players_detections.class_id == MODEL_CLASSES["team1"] ]
            team2_detections =  players_detections[players_detections.class_id == MODEL_CLASSES["team2"] ]
           
            
            players_detections = players_detections[players_detections.class_id != MODEL_CLASSES["goalkepper"]]
            # assign the ball to the closest player
            player_ind = assign_ball_to_player(players_detections,ball_detections.xyxy)
            all_players = players_detections
            if player_ind != -1:
                class_id_active_player = players_detections.class_id[player_ind]
                ball_posession[class_id_active_player]+=1
                all_players.class_id[player_ind] = MODEL_CLASSES["active_player"]
            elif class_id_active_player:
                ball_posession[class_id_active_player]+=1
            active_player_detection = all_players[all_players.class_id == MODEL_CLASSES["active_player"]]

            #  adding a padding to the ball detection and active player
            ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)
            active_player_detection.xyxy = sv.pad_boxes(xyxy=active_player_detection.xyxy, px=10)

            # add tracker teams
            team1_detections = tracker.update_with_detections(detections=team1_detections)
            team2_detections = tracker1.update_with_detections(detections=team2_detections)
            
            
            # creating labels 

            labels = {
                "labels_team1" :[f"{tracker_id}" for tracker_id in team1_detections.tracker_id],
                "labels_team2" : [f"{tracker_id}" for tracker_id in team2_detections.tracker_id],
                "labels_referee" : ["ref"] * len(referee_detections),
                "labels_gk" : ["GK"] * len(goalkeepers_detections)
                 
            }
            
            # Annotate the frame

            all_detection = {
                 "goalkeepers":goalkeepers_detections,
                 "ball":ball_detections,
                 "palyers":players_detections,
                 "referee":referee_detections,
                 "team1":team1_detections,
                 "team2":team2_detections,
                 "active_player":active_player_detection
            }
            
            # detect pitch key points

            result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
            key_points = sv.KeyPoints.from_inference(result)

            # project ball, players and referies on pitch

            filter = key_points.confidence[0] > 0.5
            frame_reference_points = key_points.xy[0][filter]
            pitch_reference_points = np.array(CONFIG.vertices)[filter]

            transformer = ViewTransformer(
                source=frame_reference_points,
                target=pitch_reference_points
            )

            frame_ball_xy = ball_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
            pitch_ball_xy = transformer.transform_points(points=frame_ball_xy)

            players_xy = players_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
            pitch_players_xy = transformer.transform_points(points=players_xy)

            referees_xy = referee_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
            pitch_referees_xy = transformer.transform_points(points=referees_xy)
            
            # visualize video game-style radar view

            annotated_frame1 = draw_pitch(CONFIG)
            annotated_frame1 = draw_points_on_pitch(
                config=CONFIG,
                xy=pitch_ball_xy,
                face_color=sv.Color.WHITE,
                edge_color=sv.Color.BLACK,
                radius=10,
                pitch=annotated_frame1)
            annotated_frame1 = draw_points_on_pitch(
                config=CONFIG,
                xy=pitch_players_xy[players_detections.class_id == 4],
                face_color=sv.Color.from_hex('00BFFF'),
                edge_color=sv.Color.BLACK,
                radius=16,
                pitch=annotated_frame1)
            annotated_frame1 = draw_points_on_pitch(
                config=CONFIG,
                xy=pitch_players_xy[players_detections.class_id == 5],
                face_color=sv.Color.from_hex('FF1493'),
                edge_color=sv.Color.BLACK,
                radius=16,
                pitch=annotated_frame1)
            annotated_frame1 = draw_points_on_pitch(
                config=CONFIG,
                xy=pitch_referees_xy,
                face_color=sv.Color.from_hex('FFD700'),
                edge_color=sv.Color.BLACK,
                radius=16,
                pitch=annotated_frame1)

#             sv.plot_image(annotated_frame1)
            
            
            annotated_frame = annotate_frames(frame,all_detection,labels,ball_posession,MODEL_CLASSES)
            # Write the annotated frame to the output video
            # Resize the pitch diagram (annotated_frame1) to match the width of the video frame
            frame_height, frame_width, _ = annotated_frame.shape
            pitch_height, pitch_width, _ = annotated_frame1.shape

            # Center the pitch diagram horizontally with black padding
            padding_left = (frame_width - pitch_width) // 2
            padding_right = frame_width - pitch_width - padding_left

            # Add padding to the pitch diagram (annotated_frame1)
            padded_pitch = cv2.copyMakeBorder(
                annotated_frame1, 
                top=0, bottom=0, 
                left=padding_left, right=padding_right, 
                borderType=cv2.BORDER_CONSTANT, 
                value=[0, 0, 0]  # Black color padding
            )
            
            combined_frame = np.vstack((annotated_frame, padded_pitch))
#             sv.plot_image(combined_frame)
            out.write(combined_frame)
#             print(combined_frame.shape)
#             if (i%10 != 0) or i<110:
#                 i+=1
#                 sv.plot_image(annotated_frame)
#             i+=1
    #     except Exception as e:
    #         print(f"Error processing frame: {e}")
    #         continue
            # break
    # Release resources
    out.release()
    # cv2.destroyAllWindows()

    print(f"Video saved as {OUT_VIDEO}")

if __name__ == '__main__':
    main()

100%|█████████▉| 749/750 [19:26<00:01,  1.56s/it]

Video saved as /kaggle/working/08fd33_4_output.mp4
